In [31]:
import numpy as np
import pandas as pd
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from itertools import product
from datetime import datetime
import random

# my own library
from backTester.BackTester import BackTester
from backTester.BackDataLoader import BackDataLoader
from conditionGenerator.ConditionGenerator import ConditionGenerator

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
backdata = pd.read_pickle('C:\\TIL\\AutoTrader\\BinanceTrader\\backdata2.pkl')

In [43]:
## 아래 꺼가. GridSearch하는 방법

params = {
    'windowMA':[20, 25, 30],
    'windowER':[15, 20, 25],
    # 'c_std':[1.5, 1.65, 1.8, 2, 2.15],
    'c_ERmean':[1.75, 2],
    'pastFirst':[8, 10],
    'pastLast':[13, 15],
    'nowFirst':[1, 3],
    'nowLast':[4, 6],
    'th_Volume':[20000, 25000, 30000],
    'volFirst':[7, 11],
    'volLast':[14, 18]

}

items = list(params.values())
keys = list(params.keys())
prods = list(product(*items))
print(len(prods))

random.seed(777)
samples = random.sample(prods, len(prods)//10)
print(len(samples))

3456
345


In [44]:
resultNum = 20
combIndexStorer = [-1]*resultNum
paramStorer = [0]*resultNum
max_n_results = [1]*resultNum

monthPair = [[1, 2], [2, 3], [3, 4], [4, 5], [7, 8], [8, 9], [9, 10]]
backdatas = dict()
for i in range(7):
    backdatas[i] = pd.read_pickle(f"BackDataList\\backdata{i}.pkl")

for combIndex, comb in enumerate(samples):
    returnList = []

    print('combIndex: ', combIndex, end=' ')
    param = dict()
    for key, value in zip(keys, comb):
        param[key] = value
        # 요기서부터 집어넣는다.
    print('Param:', list(param.values()), end=' ')

    for i in range(7):
        backdata = backdatas[i]
    
        tester = BackTester()
        # print(id(tester), end='')

        window_MA = param['windowMA']
        window_ER = param['windowER']
        backdata['MA'] = tester.get_MA(window_MA, backdata['Close'])
        backdata['std'] = tester.get_std(window_MA, backdata['Close'])
        backdata['UB'] = backdata['MA'] + 2 * backdata['std']
        backdata['U75'] = backdata['MA'] + backdata['std']
        backdata['LB'] = backdata['MA'] - 2 * backdata['std']
        backdata['L25'] = backdata['MA'] - backdata['std']
        backdata['ER'] = backdata['Close'].diff(window_ER).abs()/backdata['Close'].diff().abs().rolling(window_ER).sum()

        backdata['ERmean'] = backdata['ER'].mean()*param['c_ERmean']

        myStrg = ConditionGenerator(19, 0) # i-4, i-3, i-2, i-1, i


        myStrg.add_andCondition('Close', 'LB', '>', [param['pastFirst'], param['pastLast']])
        myStrg.add_andCondition('Close', 'LB', '<', [param['nowFirst'], param['nowLast']])
        myStrg.add_andCondition('Volume', param['th_Volume'], '<', [param['volFirst'], param['volLast']])
        myStrg.add_andCondition('ER', 'ERmean', '<', [param['volFirst'], param['volLast']])
        myStrg.add_condition('long')

        myStrg.add_andCondition('Close', 'UB', '<', [param['pastFirst'], param['pastLast']])
        myStrg.add_andCondition('Close', 'UB', '>', [param['nowFirst'], param['nowLast']])
        myStrg.add_andCondition('Volume', param['th_Volume'], '>', [param['volFirst'], param['volLast']])
        myStrg.add_andCondition('ER', 'ERmean', '>', [param['volFirst'], param['volLast']])
        myStrg.add_condition('short')

        myStrg.add_andCondition('Close', 'MA', '<', [3, 5])
        myStrg.add_andCondition('Close', 'MA', '>', [0, 2])
        myStrg.add_andCondition('_long_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', '_long_meanPrice', '<', 0, func2=lambda x:x*(1-0.03))
        myStrg.add_andCondition('_long_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', 'MA', '>', [3, 5])
        myStrg.add_andCondition('Close', 'MA', '<', [0, 2])
        myStrg.add_andCondition('_short_flag', True, '==', 0)
        myStrg.add_condition('clear')

        myStrg.add_andCondition('Close', '_short_meanPrice', '>', 0, func2=lambda x:x*(1+0.03))
        myStrg.add_andCondition('_short_flag', True, '==', 0)
        myStrg.add_condition('clear')

        # backtest
        tester.backtest_tmp(myStrg, backdata, printLog=False)

        # r = tester.finalReturn
        returnList.append(tester.finalReturn)
        print(round(tester.finalReturn, 3), end=' ')
    r = sum(returnList)/7
    std = np.std(returnList)
    for resIdx in range(resultNum):
        if r > max_n_results[resIdx]:
            try:
                max_n_results = max_n_results[:resIdx] + [r] + max_n_results[resIdx:resultNum-1]
                combIndexStorer = combIndexStorer[:resIdx] + [combIndex] + combIndexStorer[resIdx:resultNum-1]
                paramStorer = paramStorer[:resIdx] + [comb] + paramStorer[resIdx:resultNum-1]
            except:
                max_n_results = [r] + max_n_results[resIdx:resultNum-1]
                combIndexStorer = [combIndex] + combIndexStorer[resIdx:resultNum-1]
                paramStorer = [comb] + paramStorer[resIdx:resultNum-1]

        # print(', Return:', r)
        
    print('std:', std, ' Return:', r)

with open('BB_GridSearch_params.txt', 'a') as f:
    f.write(str(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))+'\n'+'-'*20)
    f.write('Keys:'+str(keys))
    for idx, res, param in zip(combIndexStorer, max_n_results, paramStorer):
        f.write('Index'+str(idx)+' - Return - '+str(res)+' - Param '+str(param))


combIndex:  0 Param: [20, 25, 1.75, 10, 15, 3, 4, 20000, 11, 18] 1.02 1.14 1.004 0.998 1.014 0.993 0.999 std: 0.048233605309255835  Return: 1.024025386857143
combIndex:  1 Param: [25, 20, 2, 10, 13, 1, 4, 20000, 11, 14] 1.007 1.137 0.998 0.983 1.005 0.978 0.994 std: 0.051060052368653394  Return: 1.0146280625714286
combIndex:  2 Param: [25, 20, 2, 10, 13, 1, 4, 20000, 7, 18] 0.999 1.13 0.992 0.99 1.019 0.98 0.997 std: 0.048161728083623755  Return: 1.0153930764285715
combIndex:  3 Param: [25, 15, 2, 10, 15, 1, 6, 30000, 7, 18] 0.991 1.136 0.996 0.996 1.001 0.992 1.003 std: 0.04912297200998362  Return: 1.0164514478571427
combIndex:  4 Param: [30, 15, 1.75, 8, 15, 1, 4, 25000, 7, 18] 0.965 1.038 1.017 1.015 1.041 0.984 0.986 std: 0.02672257306240757  Return: 1.006610861142857
combIndex:  5 Param: [20, 25, 2, 10, 15, 1, 4, 20000, 7, 18] 0.986 1.022 1.005 0.998 1.004 0.985 0.992 std: 0.012252503581756464  Return: 0.9987936257142856
combIndex:  6 Param: [30, 25, 2, 10, 13, 3, 6, 30000, 7, 18]